In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.getcwd()
os.chdir('/content/drive/My Drive/Data/')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc3

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc3) to /tmp/pip-req-build-13pl0b35
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-13pl0b35
  Running command git checkout -q 172be42f1310cdbad48941494f84d65367de4f11
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc3-cp36-none-any.whl size=91635 sha256=a3993ce001c912a46c81f4b3062ef58210b3f0f57966990ec30297d678bcd1ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-057wxf4g/wheels/2d/b8/84/5f9abbcad32fc56a0fdd5d1b8c267221abb5ec582dde4371f7
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=dbeed08c1f9ab115b079179b2b9894dce67c0f95917f29a0ee0f04e4941650d7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
!pip install autokeras

     |████████████████████████████████| 143kB 8.6MB/s 
     |████████████████████████████████| 849kB 17.7MB/s 
     |████████████████████████████████| 358kB 51.3MB/s 
     |████████████████████████████████| 36.7MB 82kB/s 
     |████████████████████████████████| 174kB 58.5MB/s 
     |████████████████████████████████| 1.1MB 49.8MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=b0481de092f679d88c75d00b7f6b20add40258fe7fe070682af880be2528ba78
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo


**Reading the data**

In [ ]:
# dataset downloaded from https://www.kaggle.com/aritrachakraborti/ipl-2020-ball-by-ball-data?select=Deliveries+IPL+2020.csv

In [48]:
# Reading IPL 2008 to 2019 data

df_Deliveries_old = pd.read_csv("Deliveries IPL 2008-2019.csv")

# Reading IPL 2020 data

df_Deliveries_20 = pd.read_csv("Deliveries IPL 2020.csv")

In [49]:
# Merging both the df

frames = [df_Deliveries_old,df_Deliveries_20]
df = pd.concat(frames)

In [50]:
df.head(1)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


**Data cleaning**

In [51]:
# Subsetting the dataset with only required features

df=df[['match_id','inning','batting_team','bowling_team','over','ball','total_runs','player_dismissed']]

df=df.rename(columns={"match_id": "id"})

In [52]:
df.head(1)

,id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,0,NaN


In [53]:
df['batting_team'].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Mumbai Indians', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Kolkata Knight Riders', 'Kings XI Punjab', 'Delhi Daredevils',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals', 'MI', 'CSK', 'DC', 'KXIP', 'RCB', 'SRH', 'RR',
       'KKR'], dtype=object)

In [54]:
# Subsetting the dataset with only consistent teams

df=df[~df.batting_team.str.contains("Pune")]
df=df[~df.batting_team.str.contains("Kochi")]
df=df[~df.batting_team.str.contains("Gujarat")]
df=df[~df.bowling_team.str.contains("Pune")]
df=df[~df.bowling_team.str.contains("Kochi")]
df=df[~df.bowling_team.str.contains("Gujarat")]

In [55]:
df.head(1)

,id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,0,NaN


In [56]:
df['batting_team'].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Delhi Daredevils', 'Kolkata Knight Riders', 'Mumbai Indians',
       'Kings XI Punjab', 'Chennai Super Kings', 'Rajasthan Royals',
       'Deccan Chargers', 'Delhi Capitals', 'MI', 'CSK', 'DC', 'KXIP',
       'RCB', 'SRH', 'RR', 'KKR'], dtype=object)

In [57]:
# renaming the team names

In [58]:
df['batting_team']=df['batting_team'].replace({'Sunrisers Hyderabad': 'SRH', 'Deccan Chargers': 'SRH','Royal Challengers Bangalore': 'RCB', 'Mumbai Indians': 'MI',
                        'Kolkata Knight Riders': 'KKR', 'Kings XI Punjab': 'KXIP','Delhi Daredevils': 'DC', 'Delhi Capitals': 'DC',
                        'Chennai Super Kings': 'CSK', 'Rajasthan Royals': 'RR'})

In [59]:
df['bowling_team']=df['bowling_team'].replace({'Sunrisers Hyderabad': 'SRH', 'Deccan Chargers': 'SRH','Royal Challengers Bangalore': 'RCB', 'Mumbai Indians': 'MI',
                        'Kolkata Knight Riders': 'KKR', 'Kings XI Punjab': 'KXIP','Delhi Daredevils': 'DC', 'Delhi Capitals': 'DC',
                        'Chennai Super Kings': 'CSK', 'Rajasthan Royals': 'RR'})

In [60]:
df['batting_team'].unique()

array(['SRH', 'RCB', 'DC', 'KKR', 'MI', 'KXIP', 'CSK', 'RR'], dtype=object)

In [61]:
df['bowling_team'].unique()

array(['RCB', 'SRH', 'DC', 'MI', 'KKR', 'KXIP', 'CSK', 'RR'], dtype=object)

In [62]:
df.head(1)

,id,inning,batting_team,bowling_team,over,ball,total_runs,player_dismissed
0,1,1,SRH,RCB,1,1,0,NaN


**Feature engineering**

In [63]:
# reducing the over values by 1 

df['over']=df['over']-1

In [64]:
# replacing the null values with 0

df = df.replace(np.nan, 0)
df.reset_index(inplace=True)
df = df.drop('index', 1)

In [65]:
# total runs scored till current ball

df['total_score']=df.groupby(['id', 'inning'])['total_runs'].apply(lambda x: x.cumsum())

# total wickets fallen in the innings

df['player_dismissed']=np.where(df['player_dismissed']==0, 0, 1)

df['total_wickets']=df.groupby(['id', 'inning'])['player_dismissed'].apply(lambda x: x.cumsum())

In [66]:
df = df.replace(np.nan, 0)
df = df.replace(np.inf, 0)

In [67]:
#total runs scored in the innings

df['total']=df.groupby(['id','inning'])['total_runs'].transform('sum')

In [68]:
# runs scored in previous 30 balls

tmp=df.groupby(['id','inning'])['total_runs'].rolling(min_periods=1, window=30).sum().reset_index()

df['prev_runs_30']=tmp[['total_runs']]

In [69]:
# wickets fallen in previous 30 balls

tmp=df.groupby(['id','inning'])['player_dismissed'].rolling(min_periods=1, window=30).sum().reset_index()

df['prev_wickets_30']=tmp[['player_dismissed']]

In [78]:
# dot balls in previous 30 balls

df['prev_30_dot_balls']=df['total_runs']
df['prev_30_dot_balls']=np.where(df['prev_30_dot_balls']==0, 1, 0)

tmp=df.groupby(['id','inning'])['prev_30_dot_balls'].rolling(min_periods=1, window=30).sum().reset_index()
df['prev_30_dot_balls']=tmp[['prev_30_dot_balls']]

In [83]:
# dot balls in previous 30 balls

df['prev_30_boundaries']=df['total_runs']
df['prev_30_boundaries']=np.where(df['prev_30_boundaries']>3, 1, 0)

tmp=df.groupby(['id','inning'])['prev_30_boundaries'].rolling(min_periods=1, window=30).sum().reset_index()
df['prev_30_boundaries']=tmp[['prev_30_boundaries']]

In [87]:
df['overs']=df['over'].astype(str) + '.' + df['ball'].astype(str)

## **Building model for first innings**

In [88]:
df_first_innings=df.copy()

In [89]:
df_first_innings=df_first_innings.loc[df_first_innings['inning'] == 1]

In [90]:
df_first_innings=pd.get_dummies(data=df_first_innings, columns=['batting_team', 'bowling_team'])

In [91]:
df_first_innings.columns

Index(['id', 'inning', 'over', 'ball', 'total_runs', 'player_dismissed',
       'total_score', 'total_wickets', 'total', 'prev_runs_30',
       'prev_wickets_30', 'prev_30_dot_balls', 'prev_30_boundaries', 'overs',
       'batting_team_CSK', 'batting_team_DC', 'batting_team_KKR',
       'batting_team_KXIP', 'batting_team_MI', 'batting_team_RCB',
       'batting_team_RR', 'batting_team_SRH', 'bowling_team_CSK',
       'bowling_team_DC', 'bowling_team_KKR', 'bowling_team_KXIP',
       'bowling_team_MI', 'bowling_team_RCB', 'bowling_team_RR',
       'bowling_team_SRH'],
      dtype='object')

In [92]:
df_first_innings=df_first_innings[['id','batting_team_CSK', 'batting_team_DC','batting_team_KKR', 'batting_team_KXIP', 'batting_team_MI',
                   'batting_team_RCB', 'batting_team_RR', 'batting_team_SRH','bowling_team_CSK', 'bowling_team_DC', 'bowling_team_KKR',
                  'bowling_team_KXIP', 'bowling_team_MI', 'bowling_team_RCB','bowling_team_RR', 'bowling_team_SRH',
                   'overs','total_score','total_wickets','prev_runs_30','prev_wickets_30','prev_30_dot_balls','prev_30_boundaries','total']]

In [93]:
len(df_first_innings)

83691

In [94]:
from sklearn.model_selection import train_test_split

X = df_first_innings.drop(labels=['total','id'], axis=1)
y = df_first_innings['total'].values

In [95]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42,stratify=y)

In [96]:
X_train=X_train.values
X_test=X_test.values
X_train=np.asarray(X_train).astype(np.float32)
X_test=np.asarray(X_test).astype(np.float32)

In [97]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(62768, 23) (20923, 23) (62768,) (20923,)


**Keras Regression Model for first innings**

In [98]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.metrics import mean_absolute_error

In [99]:
model = Sequential()

# The Input Layer :
model.add(Dense(256, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(64, kernel_initializer='normal',activation='relu'))
model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 49,409
Trainable params: 49,409
Non-trainable params: 0
_________________________________________________________________


In [100]:
checkpoint_name = 'Weights-first_innings.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_mean_absolute_error', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [101]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split = 0.3, callbacks=callbacks_list)

Epoch 1/20
1357/1374 [============================>.] - ETA: 0s - loss: 25.4142 - mean_absolute_error: 25.4142
Epoch 00001: val_mean_absolute_error improved from inf to 19.65417, saving model to Weights-first_innings.hdf5
1374/1374 [==============================] - 5s 4ms/step - loss: 25.3146 - mean_absolute_error: 25.3146 - val_loss: 19.6542 - val_mean_absolute_error: 19.6542
Epoch 2/20
1360/1374 [============================>.] - ETA: 0s - loss: 16.6208 - mean_absolute_error: 16.6208
Epoch 00002: val_mean_absolute_error improved from 19.65417 to 19.07187, saving model to Weights-first_innings.hdf5
1374/1374 [==============================] - 4s 3ms/step - loss: 16.6118 - mean_absolute_error: 16.6118 - val_loss: 19.0719 - val_mean_absolute_error: 19.0719
Epoch 3/20
1364/1374 [============================>.] - ETA: 0s - loss: 16.2217 - mean_absolute_error: 16.2217
Epoch 00003: val_mean_absolute_error improved from 19.07187 to 16.91282, saving model to Weights-first_innings.hdf5
1374/1

In [102]:
# Load wights file of the best model :
wights_file = 'Weights-first_innings.hdf5' # choose the best checkpoint 
model.load_weights(wights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [103]:
prediction=model.predict(X_test)
mean_absolute_error (y_test, prediction)

14.304937143612506

**AutoKeras regression model for first innings**

In [104]:
from autokeras import StructuredDataRegressor

In [107]:
# define the search
search = StructuredDataRegressor(max_trials=5, loss='mae',metrics=["mae"])

# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [108]:
# get the best performing model
first_innings_model = search.export_model()

In [109]:
# summarize the loaded model
first_innings_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 23)                0         
_________________________________________________________________
normalization (Normalization (None, 23)                47        
_________________________________________________________________
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

In [110]:
prediction=first_innings_model.predict(X_test)
mean_absolute_error (y_test, prediction)

13.312941613334296

**Building model for second innings**

In [142]:
df_second_innings=df.copy()

In [143]:
# first innings total

df_second_innings['1st_innings_total']=df_second_innings.groupby(['id'])['total'].transform('first')

In [144]:
df_second_innings=df_second_innings.loc[df_second_innings['inning'] == 2]

In [145]:
df_second_innings=pd.get_dummies(data=df_second_innings, columns=['batting_team', 'bowling_team'])

In [146]:
df_second_innings=df_second_innings[['id','inning','batting_team_CSK', 'batting_team_DC','batting_team_KKR', 'batting_team_KXIP', 'batting_team_MI',
                   'batting_team_RCB', 'batting_team_RR', 'batting_team_SRH','bowling_team_CSK', 'bowling_team_DC', 'bowling_team_KKR',
                  'bowling_team_KXIP', 'bowling_team_MI', 'bowling_team_RCB','bowling_team_RR', 'bowling_team_SRH',
                   'overs','total_score','total_wickets','prev_runs_30','prev_wickets_30','prev_30_dot_balls','prev_30_boundaries','1st_innings_total','total']]

In [147]:
# Splitting the data into train and test set
X = df_second_innings.drop(labels=['total','id'], axis=1)
y = df_second_innings['total'].values

In [148]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42,stratify=y)

In [149]:
X_train=X_train.values
X_test=X_test.values
X_train=np.asarray(X_train).astype(np.float32)
X_test=np.asarray(X_test).astype(np.float32)

In [150]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(58589, 25) (19530, 25) (58589,) (19530,)


In [177]:
second_innings = Sequential()

# The Input Layer :
second_innings.add(Dense(256, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
second_innings.add(Dense(128, kernel_initializer='normal',activation='relu'))
second_innings.add(Dense(64, kernel_initializer='normal',activation='relu'))
second_innings.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
second_innings.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
second_innings.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
second_innings.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               6656      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 49,921
Trainable params: 49,921
Non-trainable params: 0
_________________________________________________________________


In [178]:
checkpoint_name = 'Weights-second_innings.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_mean_absolute_error', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [179]:
second_innings.fit(X_train, y_train, epochs=20, batch_size=32, validation_split = 0.3, callbacks=callbacks_list)

Epoch 1/20
1274/1282 [============================>.] - ETA: 0s - loss: 16.9810 - mean_absolute_error: 16.9810
Epoch 00001: val_mean_absolute_error improved from inf to 14.09812, saving model to Weights-second_innings.hdf5
1282/1282 [==============================] - 5s 4ms/step - loss: 16.9570 - mean_absolute_error: 16.9570 - val_loss: 14.0981 - val_mean_absolute_error: 14.0981
Epoch 2/20
1281/1282 [============================>.] - ETA: 0s - loss: 13.0250 - mean_absolute_error: 13.0250
Epoch 00002: val_mean_absolute_error improved from 14.09812 to 12.29185, saving model to Weights-second_innings.hdf5
1282/1282 [==============================] - 4s 3ms/step - loss: 13.0278 - mean_absolute_error: 13.0278 - val_loss: 12.2918 - val_mean_absolute_error: 12.2918
Epoch 3/20
1277/1282 [============================>.] - ETA: 0s - loss: 12.3961 - mean_absolute_error: 12.3961
Epoch 00003: val_mean_absolute_error improved from 12.29185 to 11.33555, saving model to Weights-second_innings.hdf5
128

In [180]:
# Load wights file of the best model :
wights_file = 'Weights-second_innings.hdf5' # choose the best checkpoint 
second_innings.load_weights(wights_file) # load it
second_innings.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [181]:
prediction=second_innings.predict(X_test)
mean_absolute_error (y_test, prediction)

10.376078607898092

**AutoKeras regression model for second innings**

In [151]:
# define the search
search = StructuredDataRegressor(max_trials=7, loss='mae',metrics=["mae"])

# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

In [152]:
# get the best performing model
second_innings_model = search.export_model()

In [153]:
# summarize the loaded model
second_innings_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 25)                0         
_________________________________________________________________
normalization (Normalization (None, 25)                51        
_________________________________________________________________
dense (Dense)                (None, 256)               6656      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

In [154]:
prediction=second_innings_model.predict(X_test)
mean_absolute_error (y_test, prediction)

22.784962175079205

In [ ]:
# For the first innings the autokeras model is giving better results

# For the second innings the squential model is giving better results

# ***Score Prediction***

In [215]:
# Function for first innings score prediction :

def first_innings_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries):

	temp_array = list()

	if Bat_Team == 'CSK':
		temp_array = temp_array + [1,0,0,0,0,0,0,0]
	elif Bat_Team == 'DC':
		temp_array = temp_array + [0,1,0,0,0,0,0,0]
	elif Bat_Team == 'KKR':
		temp_array = temp_array + [0,0,1,0,0,0,0,0]
	elif Bat_Team == 'KXIP':
		temp_array = temp_array + [0,0,0,1,0,0,0,0]
	elif Bat_Team == 'MI':
		temp_array = temp_array + [0,0,0,0,1,0,0,0]
	elif Bat_Team == 'RCB':
		temp_array = temp_array + [0,0,0,0,0,1,0,0]
	elif Bat_Team == 'RR':
		temp_array = temp_array + [0,0,0,0,0,0,1,0]
	elif Bat_Team == 'SRH':
		temp_array = temp_array + [0,0,0,0,0,0,0,1]
		
	if Bowl_Team == 'CSK':
		temp_array = temp_array + [1,0,0,0,0,0,0,0]
	elif Bowl_Team == 'DC':
		temp_array = temp_array + [0,1,0,0,0,0,0,0]
	elif Bowl_Team == 'KKR':
		temp_array = temp_array + [0,0,1,0,0,0,0,0]
	elif Bowl_Team == 'KXIP':
		temp_array = temp_array + [0,0,0,1,0,0,0,0]
	elif Bowl_Team == 'MI':
		temp_array = temp_array + [0,0,0,0,1,0,0,0]
	elif Bowl_Team == 'RCB':
		temp_array = temp_array + [0,0,0,0,0,1,0,0]
	elif Bowl_Team == 'RR':
		temp_array = temp_array + [0,0,0,0,0,0,1,0]
	elif Bowl_Team == 'SRH':
		temp_array = temp_array + [0,0,0,0,0,0,0,1]
		
	temp_array = temp_array + [overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries]
	
	data = np.array([temp_array])
	
	my_prediction = int(first_innings_model.predict(data))
	
	print('Predicted score is from',my_prediction-13,'to',my_prediction+13)

In [229]:
# Function for second innings score prediction :

def second_innings_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries,first_innings_total):

	temp_array = list()

	if Bat_Team == 'CSK':
		temp_array = temp_array + [1,0,0,0,0,0,0,0]
	elif Bat_Team == 'DC':
		temp_array = temp_array + [0,1,0,0,0,0,0,0]
	elif Bat_Team == 'KKR':
		temp_array = temp_array + [0,0,1,0,0,0,0,0]
	elif Bat_Team == 'KXIP':
		temp_array = temp_array + [0,0,0,1,0,0,0,0]
	elif Bat_Team == 'MI':
		temp_array = temp_array + [0,0,0,0,1,0,0,0]
	elif Bat_Team == 'RCB':
		temp_array = temp_array + [0,0,0,0,0,1,0,0]
	elif Bat_Team == 'RR':
		temp_array = temp_array + [0,0,0,0,0,0,1,0]
	elif Bat_Team == 'SRH':
		temp_array = temp_array + [0,0,0,0,0,0,0,1]
		
	if Bowl_Team == 'CSK':
		temp_array = temp_array + [1,0,0,0,0,0,0,0]
	elif Bowl_Team == 'DC':
		temp_array = temp_array + [0,1,0,0,0,0,0,0]
	elif Bowl_Team == 'KKR':
		temp_array = temp_array + [0,0,1,0,0,0,0,0]
	elif Bowl_Team == 'KXIP':
		temp_array = temp_array + [0,0,0,1,0,0,0,0]
	elif Bowl_Team == 'MI':
		temp_array = temp_array + [0,0,0,0,1,0,0,0]
	elif Bowl_Team == 'RCB':
		temp_array = temp_array + [0,0,0,0,0,1,0,0]
	elif Bowl_Team == 'RR':
		temp_array = temp_array + [0,0,0,0,0,0,1,0]
	elif Bowl_Team == 'SRH':
		temp_array = temp_array + [0,0,0,0,0,0,0,1]
		
	temp_array =[2] + temp_array + [overs,total_score,total_wickets,prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries,first_innings_total]
	
	data = np.array([temp_array])
	
	my_prediction = int(second_innings.predict(data))
	
	print('Predicted score is from',my_prediction-11,'to',my_prediction+11)

**First innings score prediction** 

In [267]:
Bat_Team = 'KKR' # CSK,DC,KKR,KXIP,MI,RCB,RR,SRH

Bowl_Team = 'RCB' # CSK,DC,KKR,KXIP,MI,RCB,RR,SRH 

overs = 10.3

total_score = 67 # current score

total_wickets = 2 # current wicket 

prev_runs_30 = 37 # runs in last 30 balls

prev_wickets_30 = 1 # wickets in last 30 balls

prev_30_dot_balls = 8 # dots in last 30 balls

prev_30_boundaries = 5 # boundaries in last 30 balls

first_innings_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,
                         prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries)

Predicted score is from 142 to 168


**Second innings score prediction**

In [266]:
Bat_Team = 'MI' # CSK,DC,KKR,KXIP,MI,RCB,RR,SRH

Bowl_Team = 'CSK' # CSK,DC,KKR,KXIP,MI,RCB,RR,SRH 

overs = 12.3

total_score = 72 # current score

total_wickets = 2 # current wicket 

prev_runs_30 = 36 # runs in last 30 balls

prev_wickets_30 = 2 # wickets in last 30 balls

prev_30_dot_balls = 8 # dots in last 30 balls

prev_30_boundaries = 2 # boundaries in last 30 balls

first_innings_total = 168

second_innings_prediction(Bat_Team,Bowl_Team,overs,total_score,total_wickets,
                          prev_runs_30,prev_wickets_30,prev_30_dot_balls,prev_30_boundaries,first_innings_total)

Predicted score is from 147 to 169
